# This is my first expirementational code with a GPT model
 I am following a tutorial made by Andrej Karpathy and plan on adding my own stuff throughout the process to improve my gained knowledge and understanding.


### First we start off by downloading this tinyshakespeare text.
 This wil be our starting data set for the transformer.

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-14 16:22:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-03-14 16:22:49 (7.47 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r') as file:
    text = file.read()

In [2]:
print(len(text))

1115394


Here we are defining out vocabulary set. We do this by getting rid of the duplicates, then sorting it.

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Now we are doing the encoding and decoding. This is done in the following way:
we create lookup tables (dictionaries) from number to characters and vice-versa.
Then when encoding, we find the index of the characters in the given string, and return the array of indices.
As expected, for decoding, we get the collection of characters and join them together according to the given array of indices.

In [4]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    return ''.join([itos[i] for i in l])

In [5]:
encoded_hello = encode("hello")
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)


[46, 43, 50, 50, 53]
hello


In our case, we are using a very simple technique/method. This has its tradeoffs. This simplicity will make it easier to get a "finished" product, and results in easier and less complex encode and decode functions. However, there are alternatives that use much larger vocabularies (using subphrases - I believe) that results in a smaller result when encoding. I am assuming that this results in an improvement to the speed of the model at the trade off of complexity and a larger vocabulary size (storage size).

In [6]:
%pip install torch


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch

/Volumes/2Tb-ext/Coding Projects/Python/AI/Gpt-model/venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


Now, we are encoding all the text in the little shakespeare dataset.


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Let's split into a training and validation data set now.

In [9]:
split = int(0.85*len(data))

train_data = data[:split]
val_data = data[split:]


In [10]:
block_size = 8

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

the point of doing it like this is: the model gets used to understanding the sequence of characters etc.
But most importantly, for generation, it is trained to start off with just 1 character of context.

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range( block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} target: {target}')

when input is tensor([18]) target: 47
when input is tensor([18, 47]) target: 56
when input is tensor([18, 47, 56]) target: 57
when input is tensor([18, 47, 56, 57]) target: 58
when input is tensor([18, 47, 56, 57, 58]) target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target: 58


In [16]:
torch.manual_seed(1337) #pointless* - just to get same numbers as the video

batch_size = 4 # this is how many sequences we will process in parallel

block_size = 8 # again, its how much the maximum length of context will be

def get_batch(split: str) -> tuple:
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Create a tensor by stacking slices of data, each slice is of length block_size
    x = torch.stack([
        data[i:i+block_size]
        for i in ix])
    # Create a tensor by stacking slices of data, each slice is of length block_size
    # but shifted by 1 position to the right to serve as the target output
    y = torch.stack([
        data[i+1: i+block_size+1]
        for i in ix])
    return x, y

xb, yb = get_batch('train')


In [17]:
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()}  | target: {target}")


inputs:
torch.Size([4, 8])
tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])
targets:
torch.Size([4, 8])
----
when input is [53]  | target: 61
when input is [53, 61]  | target: 57
when input is [53, 61, 57]  | target: 10
when input is [53, 61, 57, 10]  | target: 0
when input is [53, 61, 57, 10, 0]  | target: 20
when input is [53, 61, 57, 10, 0, 20]  | target: 43
when input is [53, 61, 57, 10, 0, 20, 43]  | target: 1
when input is [53, 61, 57, 10, 0, 20, 43, 1]  | target: 58
when input is [39]  | target: 41
when input is [39, 41]  | target: 43
when input is [39, 41, 43]  | target: 42
when input is [39, 41, 43, 42]  | target: 1
when input is [39, 41, 43, 42, 1]  | target: 58
when input is [39, 41, 43, 42, 1, 58]  | target: 46
when input is [39, 41, 43, 42, 1, 58, 46]  | target: 43
when input is [39, 41, 43, 42, 1, 58, 46, 43]  | target: 1
when input is [52]  | target: 41

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # creating a 65 x 65 table

    def forward(self, idx, targets= None):

        # so what we're doing is:
        # we pass in the index, and get that row. I assume this will be the things that usually follow the passed in encoded value?
        logits = self.token_embedding_table(idx) # (B, T, C) B = batch | T = Time | C = vocab

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets) # this figures out how well we are predicting the next character
        # targets is passed in (training data so we know expected output)
        # this however doesn't work because: targets currently is (B x T) x C but torch wants it in the form BxCxT
        # so we need to reshape logits

        return logits, loss

    def generate(self, idx, max_new_tokens) -> int:
        """ to summarize:

        this takes the passed indices, then generates tokens (max_new_tokens is the number of tokens we generate)
        and return the passed in indices with the generated ones appended on it.
        Basicaly, we give it "he" and the number 3, then expect "hello" (as an example)
        This is done using probabilities

        T is removed as we only care about going in sequence rather than moving through time arbitrarily.
        """


        # IDX is (B , T) array of indicies (current context)
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # focus on last time step
            logits = logits[:, -1, :] # gives it in the form (B, C)
            # softamx to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Get a smaple from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # add sampled to the current sequence (kind of like autocomplete?)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

m = BigramLM(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)
# expected loss = -ln(1/vocab_size)
idx = torch.zeros((1, 1), dtype=torch.long) # we pass in the zero character in our vocab (aka new line) This helps with automated text generation
# if we pass in something else, it will work more like autocorrect rather than text generation
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))
# so we are generating 100 new tokens, with the new line passed in. Then turning it into a list and decoding it from the encoded vocab calues
# right now, this is completely random so we need to train it
# right now the generate function is a bigram model, however, we still pass in all the tokens, this is so we don't have to change it later when its better.

torch.Size([32, 65])
tensor(4.9670, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [22]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)


In [28]:
batch_size = 32
for steps in range(50000):

    xb, yb= get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss.item())

2.486915111541748


In [30]:
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))



SCETULou wan th w gos lleeerdot andouedsstt:
POLowar, au; ce a ofandur llains.
ENGurimerdo I n trerd tr hor,
t O:
Bemp o wo. is merd Me annde misequ VORos o bel.
'd, hethoor f VO:
Gok h, avelemug,
QUpiben.
ARCoouro t ais obyou oung we veave, Mizeefs urorth trathr Foopam

NLORowe ht! dg
Toith wh.
T: m; nisthiturive, inaves d ay t'l t we vy masthory LO:
MENo:
Wh nalcecide.
Vourerds hatrearabenmy at Whtone jom, thesodve plll sowhe t seoon;
NThtyoowinatit fou wimy an b, m---t, LI
Se sine gsthe
HAsat


Mathematical Trick in self-attention

In [31]:
torch.manual_seed(1337)

B,T,C = 4,8,2
x = torch.randn(B,T,C)

x.shape

torch.Size([4, 8, 2])

In [33]:
# tokens should only talk to tokens in the past
# easiest way for tokens to communicate is to average over all the tokens in the past
xbow = torch.zeros((B,T,C)) # x Bag of words
for b in range(B):
    for t in range(T):
        xbow[b,t] = x[b,:t+1].mean(dim=0)
xbow.shape

# this is a very simple way to do it, but it doesn't take into account the context of the tokens


torch.Size([4, 8, 2])

In [35]:
print(x[0])

print(xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [39]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True) # to summarize, this gives us a matrix, where each row sums to 1.
# the number of non-zero elements in each row is the number of tokens in the past that the token in the row talks to.
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print(c)
print(c.shape)


tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
torch.Size([3, 2])
